In [145]:
import torch
import torch.nn as nn
import torch.nn.functional as F     

In [146]:
class CNN(nn.Module):
    def __init__(self, input_chnls, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = input_chnls, out_channels = 16, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
        self.fc1 = nn.Linear(32*62*25, 120)
        self.fc2 = nn.Linear(120, num_classes)
        self.dropout = nn.Dropout(0.2) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  
        x = self.fc2(x)
        return x


In [114]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [129]:
model = CNN(3, 100).to(device)

In [116]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
    

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transforms = None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir 
        self.transforms = transforms


    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path)
        label = self.labels_df.iloc[idx, 1]
        y_label = self.class_to_idx[label]

        if self.transforms:
            image = self.transforms(image)

        return image, y_label

In [118]:
transform = transforms.Compose([
    transforms.Resize((250, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [119]:
dataset = CustomImageDataset(
    csv_file = 'captcha_images/labels_easy.csv',
    img_dir = 'captcha_images/easy',
    transforms = transform
)

In [130]:

dataloader = DataLoader(dataset, batch_size = 4, shuffle = True)

In [131]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [132]:
criterion = nn.CrossEntropyLoss()

In [140]:
def train():
    model.train()  # Set model to training mode
    for epoch in range(100):
        running_loss = 0.0
        epoch_loss = 0.0
        num_batches = 0
        
        for i, data in enumerate(dataloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_loss += loss.item()
            num_batches += 1
            
            if i % 10 == 9:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
                running_loss = 0.0
        
        # Print epoch summary
        avg_epoch_loss = epoch_loss / num_batches
        print(f'Epoch {epoch + 1} completed. Average loss: {avg_epoch_loss:.4f}')
    
    print("Training completed!")

In [141]:
train()

[Epoch 1, Batch 10] loss: 0.001
[Epoch 1, Batch 20] loss: 0.019
Epoch 1 completed. Average loss: 0.0164
[Epoch 2, Batch 10] loss: 0.015
[Epoch 2, Batch 20] loss: 0.009
Epoch 2 completed. Average loss: 0.0732
[Epoch 2, Batch 10] loss: 0.015
[Epoch 2, Batch 20] loss: 0.009
Epoch 2 completed. Average loss: 0.0732
[Epoch 3, Batch 10] loss: 0.090
[Epoch 3, Batch 20] loss: 0.206
Epoch 3 completed. Average loss: 0.1185
[Epoch 4, Batch 10] loss: 0.095
[Epoch 3, Batch 10] loss: 0.090
[Epoch 3, Batch 20] loss: 0.206
Epoch 3 completed. Average loss: 0.1185
[Epoch 4, Batch 10] loss: 0.095
[Epoch 4, Batch 20] loss: 0.079
Epoch 4 completed. Average loss: 0.0716
[Epoch 5, Batch 10] loss: 0.017
[Epoch 5, Batch 20] loss: 0.001
[Epoch 4, Batch 20] loss: 0.079
Epoch 4 completed. Average loss: 0.0716
[Epoch 5, Batch 10] loss: 0.017
[Epoch 5, Batch 20] loss: 0.001
Epoch 5 completed. Average loss: 0.0076
[Epoch 6, Batch 10] loss: 0.015
[Epoch 6, Batch 20] loss: 0.010
Epoch 5 completed. Average loss: 0.0076


- Number of epochs as 10 - 1% accuracy
- Number of epochs as 100 - 100% accuracy
- Right now model is just memorizing, need to add more samples.